In [51]:
import numpy as np
np.set_printoptions(suppress=True, precision=4)

def set_lowVal_zero(X):
    X[np.abs(X) < 9e-15] = 0
    return X

def Householder(x, i):
    alpha = -np.sign(x[i]) * np.linalg.norm(x)
    e = np.zeros(len(x), dtype=complex)
    e[i] = 1.0
    v = (x - alpha * e)
    w = v / np.linalg.norm(v)
    P = np.eye(len(x), dtype=complex) - 2 * np.outer(w, np.conj(w))
    return P

def Golub_Kahan(X):
    m, n = X.shape
    J = X.copy().astype(complex)
    U = np.identity(m, dtype=complex)
    V = np.identity(n, dtype=complex)

    for i in range(n - 2):
        h = np.zeros(m, dtype=complex)
        h[i:] = J[i:, i]
        P = Householder(h, i)
        J = set_lowVal_zero(P @ J)
        U = U @ P

        h = np.zeros(n, dtype=complex)
        h[i+1:] = J[i, i+1:]
        Q = Householder(h, i+1)
        J = set_lowVal_zero(J @ Q)
        V = V @ Q

    return U, J, V


def givensrotation(a, b):
    z = a*a + b*b
    r = np.sqrt(z.real**2 + z.imag**2)
    theta = np.arctan2(z.imag, z.real)
    hypo = np.sqrt(r) * (np.cos(theta / 2) + 1j * np.sin(theta / 2))
    if hypo == 0:
        return 1, 0
    else:
        cos1 = a/ hypo
        sin1 = -b / hypo
        return cos1, sin1

def QR_givens(A):
    m, n = A.shape
    R = A.copy()
    Q = np.identity(m,dtype=np.complex128)
    for i in range(0, n - 1):
        for j in range(i + 1, m):
            cos, sin = givensrotation(R[i, i], R[j, i])
            R[i], R[j] = (R[i]*cos) + (R[j]*(-sin)), (R[i]*sin) + (R[j] * cos)
            Q[i], Q[j] = (Q[i]*cos) + (Q[j]*(-sin)), (Q[i]*sin) + (Q[j] * cos)
    return np.transpose(np.conj(Q)), R

def givensrotation2(a, b):
    if b == 0:
        c = 1
        s = 0
    elif np.abs(b) > np.abs(a):
        tau = -a / b
        s = 1 / np.sqrt(1 + np.abs(tau)**2)
        c = s * tau
    else:
        tau = -b / a
        c = 1 / np.sqrt(1 + np.abs(tau)**2)
        s = c * tau
    return c, s


def QR_givens2(A):
    m, n = A.shape
    R = A.copy().astype(np.complex128)
    Q = np.identity(m, dtype=np.complex128)

    for j in range(n):
        for i in range(m-1, j, -1):
            a = R[i-1, j]
            b = R[i, j]
            if np.abs(b) > 1e-12:  
                c, s = givensrotation2(a, b)

                G = np.array([[c, s], [-np.conj(s), np.conj(c)]])
                R[[i-1, i], :] = G @ R[[i-1, i], :]

                Q[:, [i-1, i]] = Q[:, [i-1, i]] @ G.conj().T

    return Q, R

def QR_iterate(A):
    n = A.shape[0]
    Q_total = np.eye(n, dtype=complex)
    A_k = A.copy().astype(complex)
    for _ in range(100):
        Q, R = np.linalg.qr(A_k)
        A_k = R @ Q
        Q_total = Q_total @ Q
    return Q_total, A_k

def eigenvalues(A):
    U,S,Vh = np.linalg.svd(A)
    eigvals = []
    for i in range(len(S)):
        Av = A @ U[:, i]
        proj = np.vdot(U[:, i], Av)
        eigvals.append(np.real_if_close(proj))

    eigvals = np.array(eigvals)
    return eigvals, U

def fix_sign(v):
    idx = np.argmax(np.abs(v))
    if np.real(v[idx]) < 0:
        v *= -1
    return v

def compute_svd2(A):
    m, n = A.shape
    AAt = A @ A.conj().T
    AtA = A.conj().T @ A

    vals1, vecs1 = eigenvalues(AAt)
    vals2, vecs2 = eigenvalues(AtA)
    print(vals1, vals2)
    # idx = np.argsort(-vals1)  
    # vals1 = vals1[idx]
    # vecs1 = vecs1[:, idx]

    # idx2 = np.argsort(-vals2)
    # vals2 = vals2[idx2]
    # vecs2 = vecs2[:, idx2]


    sigma = np.sqrt(np.clip(vals1, 0, None))
    Sigma = np.zeros((m, n), dtype=complex)
    np.fill_diagonal(Sigma, sigma)

    U = vecs1
    V = vecs2

    U = set_lowVal_zero(U)
    Sigma[np.abs(Sigma) < 1e-4] = 0
    V[np.abs(V) < 1e-4] = 0

    for i in range(min(m, n)):
        Av = A @ V[:, i]
        sig_ui = sigma[i] * U[:, i]
        phase = np.vdot(sig_ui, Av)
        if np.real(phase) < 0:
            U[:, i] *= -1
            V[:, i] *= -1
    return U, Sigma, V.conj().T

def compute_svd3(A):
    m, n = A.shape
    AAt = A @ A.conj().T
    AtA = A.conj().T @ A


    vals1, vecs1 = eigenvalues(AAt)
    vals2, vecs2 = eigenvalues(AtA)
    #vecs1 = fix_sign(vecs1)
    #vecs2 = fix_sign(vecs2)
    
    sigma = np.sqrt(np.clip(vals1, 0, None))
    V = vecs2

    U = vecs1

    Sigma = np.zeros((m, n), dtype=np.complex128)
    
    np.fill_diagonal(Sigma, sigma)
    
    print(vals1)
    print(vals2)
    
    U[np.abs(U) < 1e-4] = 0
    Sigma[np.abs(Sigma) < 1e-4] = 0
    V[np.abs(V) < 1e-4] = 0
    
    for i in range(min(m,n)):
        Av = A @ V[:, i]
        sig_ui = sigma[i] * U[:, i]
        
        if np.real(np.vdot(Av, sig_ui)) < 0:
            U[:, i] *= -1
            #V[:, i] *= -1

    return U, Sigma, V.conj().T

def qr_eigen(A, max_iter=100, tol=1e-12):
    n = A.shape[0]
    A_k = A.copy().astype(complex)
    Q_total = np.eye(n, dtype=complex)
    j = 0

    for _ in range(max_iter):
        #Q, R = np.linalg.qr(A_k)
        Q, R = QR_givens2(A_k)
        A_k_new = R @ Q
        Q_total = Q_total @ Q
        j+=1

        if np.linalg.norm(A_k_new - A_k) < tol:
            print(f"{j} iterations")
            break
        A_k = A_k_new

    eigvals = np.real(np.diag(A_k))
    return np.array(eigvals), Q_total



def compute_svd(A):
    m, n = A.shape
    AAt = A @ A.conj().T

    eigvals, U = qr_eigen(AAt)
    idx = np.argsort(-eigvals)
    eigvals = eigvals[idx]
    U = U[:, idx]

    sigma = np.sqrt(np.clip(eigvals[:min(m, n)], 0, None))
    Sigma = np.zeros((m, n), dtype=complex)
    np.fill_diagonal(Sigma, sigma)


    V = np.zeros((n, n), dtype=complex)
    for i in range(min(m, n)):
        if sigma[i] > 1e-12:
            V[:, i] = (A.conj().T @ U[:, i]) / sigma[i]
            V[:, i] /= np.linalg.norm(V[:, i])  
        else:
            V[:, i] = 0

    Vh = V.conj().T
    return U, Sigma, Vh



def pinv_from_svd(U, Sigma, Vh):
    tol=1e-10
    m, n = Sigma.shape
    sigma_vals = np.diag(Sigma)[:min(m, n)]
    sigma_inv = np.array([1/s if np.abs(s) > tol else 0 for s in sigma_vals])
    Sigma_inv = np.zeros((n, m), dtype=complex)
    for i in range(len(sigma_inv)):
        Sigma_inv[i, i] = sigma_inv[i]
    return Vh.conj().T @ Sigma_inv @ U.conj().T

def pinv(A):
    u, j, v = Golub_Kahan(A)
    U, Sigma, Vh = compute_svd(j)
    U_final = u @ U
    V_final = v @ Vh
    return pinv_from_svd(U_final, Sigma, V_final)



In [52]:
A = np.random.randn(99,2) + 1j*np.random.randn(99,2)

In [53]:
u,s,vt = compute_svd(A)

In [54]:
compute_svd(A)

(array([[ 0.0606+0.0847j, -0.0752-0.0459j, -0.0867-0.177j , ...,
         -0.0329-0.1435j,  0.0766+0.0385j, -0.1091+0.1025j],
        [-0.0865-0.0411j,  0.0707+0.0905j,  0.0628-0.0761j, ...,
         -0.0386+0.1119j, -0.0086-0.1251j,  0.0119-0.0365j],
        [-0.0013+0.0917j, -0.0405-0.0554j, -0.0518+0.0217j, ...,
         -0.0695-0.0403j, -0.124 +0.0228j,  0.0343+0.0203j],
        ...,
        [-0.1119-0.0906j,  0.1119+0.0347j,  0.0272-0.0695j, ...,
         -0.0019+0.078j , -0.0228+0.0427j, -0.1736+0.0432j],
        [-0.1088-0.0289j,  0.0909+0.071j ,  0.0696+0.0512j, ...,
          0.0526+0.0355j, -0.0033+0.1075j, -0.0139+0.0236j],
        [ 0.2251-0.0931j, -0.0738-0.0804j, -0.0543+0.0389j, ...,
          0.0367-0.01j  , -0.0553+0.098j ,  0.1021+0.0224j]]),
 array([[10.5258+0.j,  0.    +0.j],
        [ 0.    +0.j, 10.251 +0.j],
        [ 0.    +0.j,  0.    +0.j],
        [ 0.    +0.j,  0.    +0.j],
        [ 0.    +0.j,  0.    +0.j],
        [ 0.    +0.j,  0.    +0.j],
        [ 0. 

In [55]:
np.linalg.svd(A)

SVDResult(U=array([[-0.0617-0.0581j,  0.0551-0.0813j, -0.0493+0.0093j, ...,
        -0.0195+0.0879j, -0.0603+0.1024j,  0.118 -0.1086j],
       [ 0.1656+0.0998j,  0.0156-0.0669j,  0.1541-0.0842j, ...,
        -0.1553-0.0978j, -0.0538-0.0063j,  0.1101+0.0579j],
       [-0.1115-0.1331j,  0.0503+0.0275j,  0.9718-0.0015j, ...,
         0.0109+0.028j ,  0.0039+0.0087j, -0.0048-0.0198j],
       ...,
       [ 0.1886-0.0362j,  0.0085+0.0704j,  0.0123-0.027j , ...,
         0.9639+0.0003j, -0.0173+0.0025j,  0.0315+0.0042j],
       [ 0.0697+0.013j ,  0.07  +0.0845j,  0.0023-0.0091j, ...,
        -0.0175+0.0011j,  0.984 +0.0018j,  0.0237+0.0049j],
       [-0.1444+0.0234j, -0.1166-0.0783j, -0.0032+0.0191j, ...,
         0.0299-0.0082j,  0.022 -0.0099j,  0.9622+0.0037j]]), S=array([14.1762, 12.9288]), Vh=array([[-0.1747+0.j    ,  0.9586-0.2248j],
       [-0.9846+0.j    , -0.1701+0.0399j]]))

In [56]:
u@s@vt

array([[ 0.047 +0.7821j, -1.4052-0.2147j],
       [-0.5911-0.7093j,  1.343 +0.408j ],
       [-0.4064+0.5376j, -0.9504-0.7011j],
       [ 0.4296-1.0844j, -1.453 -1.0397j],
       [-0.1321+0.4268j,  0.3411-0.8398j],
       [-0.7562-1.2401j, -0.0156-0.4229j],
       [ 0.7998+1.389j , -1.2015-0.5161j],
       [-0.171 -0.2811j, -0.4541-1.4436j],
       [-0.052 +0.7257j,  0.3401+0.6738j],
       [-0.33  +0.4746j, -1.1388-0.1705j],
       [-0.9183+0.687j ,  0.5521-0.4688j],
       [ 0.4108-0.1407j,  0.2059+1.1825j],
       [-0.3974+0.4396j, -0.0708+0.5199j],
       [-0.1594-0.1238j, -0.4297+1.8518j],
       [-0.6571+1.2113j, -0.696 +0.0935j],
       [ 0.2571+0.203j , -1.1908+0.9265j],
       [ 1.0961+0.8694j, -1.3448+0.859j ],
       [ 0.1218-0.8384j,  1.0721+1.6707j],
       [ 0.9901+1.0959j,  0.3743+0.5841j],
       [-0.5917-0.6322j,  0.3805+0.5096j],
       [-0.0499-0.3843j, -0.6494-1.3443j],
       [ 0.2132+0.1491j,  0.9592+0.1219j],
       [-0.6072+0.6652j, -0.4976-0.1877j],
       [-0.

In [57]:
A

array([[-0.5487+1.1792j, -1.1035-0.3855j],
       [-0.6084+0.604j ,  2.5684+0.9836j],
       [-0.3645-0.0207j, -2.0649-1.4885j],
       [-0.9846-0.3247j, -2.1257-0.0003j],
       [-0.3175+0.1111j,  0.035 -1.2j   ],
       [-1.4751-1.6672j, -0.2531-0.2957j],
       [ 0.3664+1.4805j, -0.8069-0.0718j],
       [-0.4265-0.2392j,  0.4724-1.2243j],
       [-0.7904+0.2761j,  0.7801+1.1744j],
       [-0.2929+0.4451j, -1.7657-0.9526j],
       [ 0.1883+0.1742j,  0.6368-0.4701j],
       [ 0.3408-1.0942j,  0.7107+1.1963j],
       [-0.0734-0.6677j, -0.3685+0.6817j],
       [ 0.0871-0.0872j, -1.5803+1.7014j],
       [-0.9875+1.0143j, -0.5024+0.2813j],
       [ 0.4931-0.0541j, -1.637 +0.1252j],
       [ 1.0628+0.5965j, -1.4972+0.2662j],
       [-0.1972-0.6345j,  1.4421+1.9641j],
       [-0.4618+1.4899j,  0.9844+0.7207j],
       [ 1.1219-0.3417j,  0.3344-0.135j ],
       [-0.9311-0.9371j, -0.1186-1.5955j],
       [-0.2055+0.6583j,  0.9418+0.5213j],
       [ 0.2176+1.4821j,  1.7724-0.2448j],
       [-1.

In [58]:
def RMSE(A, B):
    return np.sqrt(np.mean(np.abs(A - B)**2))


In [59]:
A_pinv_ref = np.linalg.pinv(A)

In [60]:
A_pinv = pinv(A)

In [61]:
RMSE(A_pinv, A_pinv_ref)

0.006372731159922836

In [62]:
qn,rn = np.linalg.qr(A)

In [63]:
q2,r2 = QR_givens2(A)

In [49]:
RMSE(q2@r2, A)

6.535452873001119e-16

In [50]:
RMSE(qn@rn, A)

2.1500482944917313e-16